In [7]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np

In [8]:
img_height = 80
img_width = 80
batch_size = 10

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(None,80,80,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(6,activation ="softmax"))


ValueError: Input 0 of layer "max_pooling2d_3" is incompatible with the layer: expected ndim=4, found ndim=5. Full shape received: (None, None, 78, 78, 32)

In [6]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/train",
    #labels="inferred",
    label_mode="int",  # categorical, binary
    class_names=['1', '2', '3','4','5','6'],
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="training",
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/test",
    #labels="inferred",
    label_mode="int",  # categorical, binary
    class_names=['1', '2', '3','4','5','6'],
    color_mode="grayscale",
    batch_size=batch_size,
    image_size=(img_height, img_width),  # reshape if not in this size
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset="validation",
)


def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y


ds_train = ds_train.map(augment)

# Custom Loops
for epochs in range(10):
    for x, y in ds_train:
        # train here
        pass


model.compile(
    optimizer=keras.optimizers.Adadelta(),
    loss=[keras.losses.SparseCategoricalCrossentropy(from_logits=False),],
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=100, verbose=1)

Found 1152 files belonging to 6 classes.
Using 1037 files for training.
Found 144 files belonging to 6 classes.
Using 14 files for validation.
Epoch 1/100
104/104 [==============================] - 10s 85ms/step - loss: 0.1114 - accuracy: 0.9836
Epoch 2/100
104/104 [==============================] - 9s 88ms/step - loss: 0.1059 - accuracy: 0.9846
Epoch 3/100
104/104 [==============================] - 8s 75ms/step - loss: 0.1023 - accuracy: 0.9855
Epoch 4/100
104/104 [==============================] - 8s 74ms/step - loss: 0.0994 - accuracy: 0.9855
Epoch 5/100
104/104 [==============================] - 8s 76ms/step - loss: 0.0965 - accuracy: 0.9865
Epoch 6/100
104/104 [==============================] - 8s 77ms/step - loss: 0.0940 - accuracy: 0.9875
Epoch 7/100
104/104 [==============================] - 9s 83ms/step - loss: 0.0915 - accuracy: 0.9875
Epoch 8/100
104/104 [==============================] - 8s 75ms/step - loss: 0.0891 - accuracy: 0.9875
Epoch 9/100
104/104 [===================

104/104 [==============================] - 8s 72ms/step - loss: 0.0131 - accuracy: 1.0000
Epoch 80/100
104/104 [==============================] - 7s 71ms/step - loss: 0.0127 - accuracy: 1.0000
Epoch 81/100
104/104 [==============================] - 8s 73ms/step - loss: 0.0124 - accuracy: 1.0000
Epoch 82/100
104/104 [==============================] - 8s 72ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 83/100
104/104 [==============================] - 8s 73ms/step - loss: 0.0117 - accuracy: 1.0000
Epoch 84/100
104/104 [==============================] - 8s 72ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 85/100
104/104 [==============================] - 8s 72ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 86/100
104/104 [==============================] - 8s 74ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 87/100
104/104 [==============================] - 7s 71ms/step - loss: 0.0105 - accuracy: 1.0000
Epoch 88/100
104/104 [==============================] - 8s 73ms/step - loss: 0.0101 - 

In [5]:
model.save('test/model.h5')

In [26]:
score = model.evaluate(ds_validation)

2/2 [==============================] - 0s 17ms/step - loss: 1.0690 - accuracy: 0.7143
